In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [10]:
# Função para obter os índices de correção entre duas datas
def obter_indices_correcao(data_inicio, data_fim, codigo_serie):
    # Converte datas para o formato esperado pela API (dd/mm/aaaa)
    data_inicio_formatada = datetime.strptime(data_inicio, "%Y-%m-%d").strftime("%d/%m/%Y")
    data_fim_formatada = datetime.strptime(data_fim, "%Y-%m-%d").strftime("%d/%m/%Y")
    url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={data_inicio_formatada}&dataFinal={data_fim_formatada}'

    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição teve sucesso
        data = response.json()

        # Convertendo os dados para um DataFrame
        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df['valor'] = pd.to_numeric(df['valor'])
        return df

    except requests.exceptions.HTTPError as http_err:
        print(f'Erro HTTP ocorreu: {http_err}')  # Exibe o erro HTTP
    except requests.exceptions.RequestException as req_err:
        print(f'Outro erro ocorreu: {req_err}')  # Exibe outros erros de requisição
    except ValueError as val_err:
        print(f'Erro de valor: {val_err}')  # Exibe erros de valor, como JSONDecodeError
    return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

# Função para calcular o índice de correção
def calcular_indice_correcao(df):
    df['fator'] = 1 + df['valor'] / 100  # Converte índices percentuais em fatores
    fator_acumulado = df['fator'].prod()  # Multiplica todos os fatores
    return fator_acumulado

# Datas inicial e final
data_inicial = '2024-12-01'
data_final = '2025-01-31'
codigo_serie = '433'  # Exemplo de código de série (Índice Nacional de Preços ao Consumidor Amplo - IPCA)

# Lista para armazenar os resultados
resultados = []

# Gera períodos consecutivos
data_atual = datetime.strptime(data_inicial, "%Y-%m-%d")
data_fim = datetime.strptime(data_final, "%Y-%m-%d")

while data_atual <= data_fim:
    data_inicio = data_atual.strftime("%Y-%m-%d")
    indices_correcao = obter_indices_correcao(data_inicio, data_final, codigo_serie)
    if not indices_correcao.empty:
        indice_correcao = calcular_indice_correcao(indices_correcao)
        resultados.append({'data_inicial': data_inicio, 'indice_correcao': f"{indice_correcao:.10f}"})
    else:
        print(f"Não foi possível obter os índices de correção para o período começando em {data_inicio}.")
    data_atual += timedelta(days=1)  # Incrementa para o próximo dia

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)

# Exibir o DataFrame de resultados
df_resultados

Erro HTTP ocorreu: 502 Server Error: Bad Gateway for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json&dataInicial=11/12/2024&dataFinal=31/01/2025
Não foi possível obter os índices de correção para o período começando em 2024-12-11.


,data_inicial,indice_correcao
0,2024-12-01,1.0068083200
1,2024-12-02,1.0068083200
2,2024-12-03,1.0068083200
3,2024-12-04,1.0068083200
4,2024-12-05,1.0068083200
...,...,...
56,2025-01-27,1.0016000000
57,2025-01-28,1.0016000000
58,2025-01-29,1.0016000000
59,2025-01-30,1.0016000000
